In [1]:
#importar librerias importantes

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
#Lectura de Datos y presentación

datos=pd.read_excel('Base de datos 2.0.xlsx', header=0, error_bad_lines=False)
datos.head()

,Estraus,Comuna,Comercio,Educación,Habitación,Industria (1),Servicios (2),Otros (3),Sin Considerar (3),Total,...,min dist metro,Porcentaje proposito trabajo,porcentaje proposito estudio,porcentaje proposito volver a la casa,porcentaje ir de compras,porcentaje modo privado,porcentaje modo publico,porcentaje no motorizado,Área total (m2),sector
0,1,Independencia,12650,6333,78152,30140,8368,7563,40252,183458,...,3.041252,14.4932,8.9110,48.1465,8.4019,27.6373,23.7660,40.0923,4.678599e+05,norte
1,2,Independencia,39228,37662,321692,30722,19166,23144,33313,504927,...,1.609982,18.3619,8.5748,47.6031,13.9849,20.6951,30.0959,45.9323,1.134592e+06,norte
2,3,Independencia,46808,8041,213874,66033,12047,10053,35719,392575,...,2.549236,15.2141,5.9068,55.9761,8.4050,25.3639,11.8234,60.9220,9.133002e+05,norte
3,4,Independencia,32304,10895,247644,24324,14220,12453,47382,389222,...,1.620016,21.5194,8.7828,44.8588,16.4969,23.7714,23.5462,48.6555,7.513172e+05,norte
4,5,Independencia,11346,4685,59839,53861,6338,2742,30640,169451,...,3.120056,13.8301,0.0000,78.9652,4.8030,12.0075,21.1816,57.0884,4.106979e+05,norte


In [3]:
#Transformación Variables Categóricas
datoscat= datos[['sector']]
datoscat=pd.get_dummies(datoscat, columns = ['sector'], drop_first = True)
datoscat.head(10)

,sector_norte,sector_oriente,sector_poniente,sector_sur,sector_sur-oriente
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [4]:
#Estandarizacion Variables continuas
datoscont =datos[['Comercio','Educación','Habitación','Industria (1)','Servicios (2)','Otros (3)',
                  'Sin Considerar (3)','Min dist hosp (km)','Min dist esc',
                 'Min dist autopista','min dist cc','min dist metro','Porcentaje proposito trabajo',
                 'porcentaje proposito estudio','porcentaje ir de compras','porcentaje modo privado',
                  'porcentaje modo publico','porcentaje no motorizado','Área total (m2)']]
scaler=preprocessing.StandardScaler()
datos_est=scaler.fit_transform(datoscont)
datos_cont_est=pd.DataFrame({'Comercio':datos_est[:,0],'Educación':datos_est[:,1],'Habitación':datos_est[:,2],'Industria':datos_est[:,3],
                  'Servicios':datos_est[:,4],'Otros':datos_est[:,5],'Sin Considerar':datos_est[:,6],
                  'Dist_Hosp':datos_est[:,7],'Dist_Esc':datos_est[:,8],'Dist_Autopista':datos_est[:,9],
                  'Dist_CC':datos_est[:,10],'Dist_Metro':datos_est[:,11],'Prop_Trabajo':datos_est[:,12],
                  'Prop_Estudio':datos_est[:,13],'Prop_Compras':datos_est[:,14],'Modo_Privado':datos_est[:,15],
                  'Modo_Público':datos_est[:,16],'Modo_NoMotorizado':datos_est[:,17],'Area':datos_est[:,18]})
datos_cont_est.head()

,Comercio,Educación,Habitación,Industria,Servicios,Otros,Sin Considerar,Dist_Hosp,Dist_Esc,Dist_Autopista,Dist_CC,Dist_Metro,Prop_Trabajo,Prop_Estudio,Prop_Compras,Modo_Privado,Modo_Público,Modo_NoMotorizado,Area
0,-0.213384,-0.315154,-0.711652,0.238135,-0.186970,-0.148238,-0.083052,-0.346437,-0.332790,-0.770076,0.753861,0.120549,-0.120446,0.249972,-0.330194,-0.041228,-0.548257,0.108071,-0.174451
1,0.543424,1.721085,0.615364,0.250041,-0.024774,0.910651,-0.146974,-0.771294,-0.268814,-0.082552,0.292602,-0.393853,0.153017,0.199148,0.482824,-0.455163,0.015938,0.470327,-0.103766
2,0.759265,-0.204142,0.027878,0.972398,-0.131708,0.020983,-0.124810,-0.621059,0.067880,-0.587749,0.278715,-0.056283,-0.069488,-0.204173,-0.329742,-0.176781,-1.612720,1.400142,-0.127226
3,0.346264,-0.018646,0.211886,0.119158,-0.099067,0.184088,-0.017370,-1.000202,-0.616679,-0.330432,-0.279046,-0.390246,0.376208,0.230592,0.848631,-0.271736,-0.567848,0.639248,-0.144399
4,-0.250515,-0.422267,-0.811437,0.723395,-0.217462,-0.475875,-0.171598,-0.513320,-0.192178,-0.893480,0.437231,0.148872,-0.167318,-1.097102,-0.854279,-0.973169,-0.778608,1.162343,-0.180511


In [5]:
#Creación de Matriz de Caracteristicas y Variable Respuesta

X=pd.concat([datoscat,datos_cont_est],axis=1)
y =datos[['Comercio 2014']]

In [6]:
#Fijación de hiperparámetros

parametros= {'kernel': ['linear','poly','rbf','sigmoid'], 'C': [0.1,1,10,100], 'epsilon': [0.001,0.01,0.1,1],
             'shrinking': [False]}

In [7]:
#Busqueda Grid Search CV

SVM_GridSearch= GridSearchCV(SVR(), parametros, verbose=10000, cv=10, scoring='r2')
SVM_GridSearch.fit(X,y)

Fitting 10 folds for each of 64 candidates, totalling 640 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............
[CV]  C=0.1, epsilon=0.001, kernel=linear, shrinking=False, score=-0.153, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............
[CV]  C=0.1, epsilon=0.001, kernel=linear, shrinking=False, score=-0.036, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............


C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

[CV]  C=0.1, epsilon=0.001, kernel=linear, shrinking=False, score=-0.111, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............
[CV]  C=0.1, epsilon=0.001, kernel=linear, shrinking=False, score=-0.274, total=   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............
[CV]  C=0.1, epsilon=0.001, kernel=linear, shrinking=False, score=-0.717, total=   0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............
[CV]  C=0.1, epsilon=0.001, kernel=linear, shrinking=False, score=-0.158, total=   0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=linear, shrinking=False ............
[CV]  C=0.1, epsilon=0.001, kernel=linear, s

C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

[CV]  C=0.1, epsilon=0.001, kernel=poly, shrinking=False, score=-0.039, total=   0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=poly, shrinking=False ..............
[CV]  C=0.1, epsilon=0.001, kernel=poly, shrinking=False, score=-0.113, total=   0.0s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.4s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=poly, shrinking=False ..............
[CV]  C=0.1, epsilon=0.001, kernel=poly, shrinking=False, score=-0.276, total=   0.0s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.4s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=poly, shrinking=False ..............
[CV]  C=0.1, epsilon=0.001, kernel=poly, shrinking=False, score=-0.720, total=   0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.4s remaining:    0.0s
[CV] C=0.1, epsilon=0.001, kernel=poly, shrinking=False ..............
[CV]  C=0.1, epsilon=0.001, kernel=poly, shrinking=F

...

[CV]  C=100, epsilon=1, kernel=rbf, shrinking=False, score=-0.039, total=   0.0s
[Parallel(n_jobs=1)]: Done 627 out of 627 | elapsed:   19.4s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=rbf, shrinking=False ...................
[CV]  C=100, epsilon=1, kernel=rbf, shrinking=False, score=-0.001, total=   0.0s
[Parallel(n_jobs=1)]: Done 628 out of 628 | elapsed:   19.5s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=rbf, shrinking=False ...................
[CV]  C=100, epsilon=1, kernel=rbf, shrinking=False, score=-0.142, total=   0.0s
[Parallel(n_jobs=1)]: Done 629 out of 629 | elapsed:   19.5s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=rbf, shrinking=False ...................
[CV]  C=100, epsilon=1, kernel=rbf, shrinking=False, score=-0.076, total=   0.0s
[Parallel(n_jobs=1)]: Done 630 out of 630 | elapsed:   19.6s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=sigmoid, shrinking=False ...............
[CV]  C=100, epsilon=1, kernel=sigmoid, shrinking=False, score=-0.064, t

C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

[CV]  C=100, epsilon=1, kernel=sigmoid, shrinking=False, score=-0.036, total=   0.0s
[Parallel(n_jobs=1)]: Done 633 out of 633 | elapsed:   19.7s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=sigmoid, shrinking=False ...............
[CV]  C=100, epsilon=1, kernel=sigmoid, shrinking=False, score=-0.135, total=   0.0s
[Parallel(n_jobs=1)]: Done 634 out of 634 | elapsed:   19.7s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=sigmoid, shrinking=False ...............
[CV]  C=100, epsilon=1, kernel=sigmoid, shrinking=False, score=-0.602, total=   0.0s
[Parallel(n_jobs=1)]: Done 635 out of 635 | elapsed:   19.7s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=sigmoid, shrinking=False ...............
[CV]  C=100, epsilon=1, kernel=sigmoid, shrinking=False, score=-0.013, total=   0.0s
[Parallel(n_jobs=1)]: Done 636 out of 636 | elapsed:   19.8s remaining:    0.0s
[CV] C=100, epsilon=1, kernel=sigmoid, shrinking=False ...............
[CV]  C=100, epsilon=1, kernel=sigmoid, shrinking=False,

C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Cristobal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

GridSearchCV(cv=10, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100],
                         'epsilon': [0.001, 0.01, 0.1, 1],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'shrinking': [False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=10000)

In [8]:
#Mejores hiperparametros
SVM_GridSearch.best_params_

{'C': 100, 'epsilon': 1, 'kernel': 'linear', 'shrinking': False}

In [9]:
#Resultados GridSearch
SVM_GridSearch.cv_results_

{'mean_fit_time': array([0.03165703, 0.02308695, 0.02968526, 0.02867913, 0.02068565,
        0.02059002, 0.02748535, 0.02927876, 0.01879225, 0.0204905 ,
        0.02788138, 0.02748203, 0.02308908, 0.02298484, 0.0283838 ,
        0.02748094, 0.02069485, 0.02286265, 0.03348083, 0.03151667,
        0.01908078, 0.02364526, 0.02619042, 0.02678773, 0.02059636,
        0.02029777, 0.0273829 , 0.02718148, 0.01901474, 0.0204896 ,
        0.026983  , 0.027686  , 0.02008791, 0.02079117, 0.0275799 ,
        0.02708545, 0.01908834, 0.02118719, 0.02727964, 0.02838411,
        0.01888773, 0.02048714, 0.02738731, 0.02668302, 0.02009387,
        0.02658694, 0.03507793, 0.03967576, 0.03098297, 0.02329092,
        0.0297821 , 0.02828455, 0.03167727, 0.02438917, 0.02828631,
        0.029176  , 0.0275836 , 0.0268888 , 0.02978058, 0.03158245,
        0.02837949, 0.02410016, 0.03277886, 0.02828503]),
 'std_fit_time': array([0.03594757, 0.00445901, 0.00295965, 0.00232212, 0.00264432,
        0.00215372, 0.001

In [14]:
#Implementación Modelo

model= SVR(C=100, epsilon=1, kernel= 'linear', shrinking=False)


In [15]:
#Cross Validation

Scores = cross_val_score(model, X, y.values.ravel(), cv=10, scoring=('r2'))
Scores

array([0.46994162, 0.60429209, 0.38749887, 0.56304626, 0.38489108,
       0.59212244, 0.22710688, 0.34522737, 0.32379178, 0.27040881])